## AIM: Fine-tune pretrained ShallowNet as multiclass classification model for predicting the psychiatric diagnosis based on SSL features

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
import pickle
import os
from sklearn.model_selection import StratifiedGroupKFold
import contextlib
import io

import optuna

import torch.nn as nn
from lightning.pytorch.loggers import CSVLogger
import lightning.pytorch as pl
import torch
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, BatchNorm1d, Dropout, Dropout1d, ModuleList
from torch.utils.data import Dataset, DataLoader, Subset
import lightning.pytorch as pl
from torchmetrics import F1Score, ConfusionMatrix
from sklearn.metrics import f1_score, classification_report, confusion_matrix, ConfusionMatrixDisplay
from lightning.pytorch.tuner import Tuner
from lightning.pytorch import Trainer
from lightning.pytorch.callbacks import ModelCheckpoint, TQDMProgressBar, RichProgressBar
from lightning.pytorch.callbacks.early_stopping import EarlyStopping

from optuna.visualization.matplotlib import plot_contour
from optuna.visualization.matplotlib import plot_edf
from optuna.visualization.matplotlib import plot_intermediate_values
from optuna.visualization.matplotlib import plot_optimization_history
from optuna.visualization.matplotlib import plot_parallel_coordinate
from optuna.visualization.matplotlib import plot_param_importances
from optuna.visualization.matplotlib import plot_rank
from optuna.visualization.matplotlib import plot_slice
from optuna.visualization.matplotlib import plot_timeline

## Functions:

In [2]:
def load_file(file):
    with open(file, 'rb') as handle:
        return pickle.load(handle)
# functions for loading the epoched EEG data
def get_filepath(epoch_dir, participant_ids):
    """
    Function to get the filepath of the epoched EEG recording
    :param epoch_dir: directory containing the epoched EEG recordings
    :param participant_ids: list of participant IDs to include
    """
    filepaths = []
    found_ids = set()
    
    for subdir, dirs, files in os.walk(epoch_dir):
        for file in files:
            for participant_id in participant_ids:
                if participant_id in file:
                    filepaths.append(os.path.join(subdir, file))
                    found_ids.add(participant_id)
    
    # Print participant IDs if no files matching those IDs are found
    for participant_id in participant_ids:
        if participant_id not in found_ids:
            print(f"No files found for participant ID {participant_id}")

    return filepaths

class EpochDataset(torch.utils.data.Dataset):
    def __init__(self, participant_ids, epoch_dir, df_classes):
        self.filepaths = get_filepath(epoch_dir, participant_ids)
        self.epochs = []
        self.participant_ids = []
        self.diagnosis = []
        self.df_classes = df_classes
        self._load_data()
        assert len(self.epochs) == len(self.participant_ids) == len(self.diagnosis)
        print(f"Loaded {len(self.epochs)} epochs")

    def _load_data(self):
        all_epochs = []
        for filepath in self.filepaths:
            epochs = torch.load(filepath)
            # get participant ID from filepath to make sure the participant ID is correct
            participant_id = filepath.split("\\")[-1].split(".")[0]
            # get the diagnosis of the participant
            diagnosis = self.df_classes[self.df_classes["participants_ID"] == participant_id]["labels"].values[0]

            all_epochs.append(epochs)
            self.participant_ids.extend([participant_id]*epochs.shape[0])
            self.diagnosis.extend([diagnosis]*epochs.shape[0])

        self.epochs = np.concatenate(all_epochs, axis=0)

    def __len__(self):
        return self.epochs.shape[0]
    
    def __getitem__(self, idx):
        epoch = self.epochs[idx]
        y = self.diagnosis[idx]
        participant_id = self.participant_ids[idx]
        return torch.tensor(epoch, dtype=torch.float32), y, participant_id
    
def transfer_freeze_weights(pretrained_weights, model, freeze=True):
            """
            Function to transfer the pretrained weights to the downstream model
            param: pretrained_weights: the weights to transfer in a dictionary
            param: model: the model to transfer the weights to
            param: freeze: whether to freeze the pretrained weights or not
            """
            pretrained_model = model
            modified_keys = {}
            for k, v in pretrained_weights.items():
                decomposed_key = k.split('.')
                if decomposed_key[0] == 'EEGNet' or decomposed_key[0] == 'ShallowNet': # remove the first part of each key to match the model's keys
                    pretrained_key = '.'.join(decomposed_key[1:])
                    modified_keys[pretrained_key] = v

            pretrained_model.load_state_dict(modified_keys, strict=False)
            if freeze: # freeze only the layers that have been pretrained
                for name, param in pretrained_model.named_parameters():
                    if name in modified_keys.keys():
                        # print(f'Freezing layer: {name}')
                        param.requires_grad = False
            return pretrained_model

def evaluate_best_model(test_scores, best_outer_hyperparameters, checkpoints_path):
    """
    Function to evaluate the best model based on the test scores
    :param test_scores: the test scores for each fold
    :param best_outer_hyperparameters: the best hyperparameters for each fold
    """
    best_fold = np.argmax(test_scores)
    best_params = best_outer_hyperparameters[best_fold]
    print(f'Best fold: {best_fold}')
    print(f'Best hyperparameters: {best_params}')
    print(f'Best test f1-score: {test_scores[best_fold]}')

    # load the best model
    fc_layers=best_params['fc_layers']
    dropout=best_params['dropout']
    optimizer_name=best_params['optimizer']
    learning_rate=best_params['learning_rate']

    best_model_path = os.path.join(checkpoints_path, f'fold-{best_fold}-last.ckpt')
    print(f"Best model checkpoint path: {best_model_path}")
    best_model = ShallowNet.load_from_checkpoint(
        checkpoint_path=best_model_path,
        fc_layers=fc_layers,
        dropout=dropout,
        optimizer_name=optimizer_name,
        learning_rate=learning_rate,
    )

    # obtain classification report and confusion matrix on all data
    dataloader = DataLoader(dataset, batch_size=len(dataset), shuffle=False)
    trainer = Trainer(accelerator='gpu', devices=1, enable_progress_bar=False, enable_model_summary=False, logger=False, enable_checkpointing=False)
    y_hat = trainer.predict(best_model, dataloaders=dataloader)
    y_true = np.array(dataset.diagnosis)
    y_hat = np.concatenate(y_hat).flatten()
    print(y_hat.shape)
    print(classification_report(y_true, y_hat, target_names=le.classes_))
    # Create a figure with two subplots
    fig, axes = plt.subplots(1, 2, figsize=(12, 6))

    # Plot the first confusion matrix
    cm = confusion_matrix(y_true, y_hat)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=le.classes_)
    disp.plot(cmap='Blues', values_format='d', colorbar=False, ax=axes[0])
    axes[0].set_title('Confusion Matrix')

    # Plot the normalized confusion matrix
    cm_norm = confusion_matrix(y_true, y_hat, normalize='true')
    disp_norm = ConfusionMatrixDisplay(confusion_matrix=cm_norm, display_labels=le.classes_)
    disp_norm.plot(cmap='Blues', colorbar=False, ax=axes[1])
    axes[1].set_title('Normalized Confusion Matrix')
    axes[1].grid(False)

    # Adjust layout
    plt.tight_layout()
    plt.show()
    return

In [3]:
def nested_cv_optuna(n_epochs, n_trials, dataset, labels, groups, pretrained_weights):
    '''	
    Function to perform nested cross-validation with hyperparameter optimization for ShallowNet using Optuna
    :param n_epochs: number of epochs to train the model
    :param n_trials: number of trials for the hyperparameter optimization
    :param dataset: the dataset containing the EEG epochs
    :param labels: the labels of the dataset
    :param groups: the participant IDs of the dataset
    :param pretrained_weights: the pretrained weights to transfer to the model
    :return: the test scores and the best hyperparameters for each fold
    '''
    outer_cv = StratifiedGroupKFold(n_splits=5, shuffle=False)
    test_scores = []
    best_outer_hyperparameters = {}
    logger = CSVLogger("logs", name='ShallowNetTest') # log results to csv file
    torch.set_float32_matmul_precision('high') # for use of tensor cores
    for i, (train_val_idx, test_idx) in enumerate(outer_cv.split(dataset, labels, groups)):
        # print(f'Size of train/val set: {len(train_val_idx)}')
        # print(f'Size of test set: {len(test_idx)}')
        train_val_set = Subset(dataset, train_val_idx)
        test_set = Subset(dataset, test_idx)
        
        # inner cv
        def objective_cv(trial):
            # suggest values for the hyperparameters
            learning_rate = trial.suggest_float('learning_rate', 1e-5, 1e-1, log=True)
            fc_layers = trial.suggest_int('fc_layers', 1, 5)
            optimizer_name = trial.suggest_categorical('optimizer', ['Adam', 'RMSprop', 'SGD'])
            batch_size = trial.suggest_categorical('batch_size', [50, 100, 178, 350, 476, 700])
            dropout = trial.suggest_float('dropout', 0, 0.8)
            hyperparameters = dict(learning_rate=learning_rate, fc_layers=fc_layers, optimizer_name=optimizer_name,
                                batch_size=batch_size, dropout=dropout)
            
            # n_epochs = batch_size * 4 # vary training epochs based on batch size

            # create the model
            model = ShallowNet(fc_layers=fc_layers, dropout=dropout, optimizer_name=optimizer_name, learning_rate=learning_rate)

            pretrained_model = transfer_freeze_weights(pretrained_weights, model, freeze=True)
            
            # inner cv
            inner_cv = StratifiedGroupKFold(n_splits=3, shuffle=False)
            val_scores = []
            for j, (train_idx, val_idx) in enumerate(inner_cv.split(train_val_set, labels[train_val_idx], groups[train_val_idx])):
                # print(f'Size of train set: {len(train_idx)}')
                # print(f'Size of val set: {len(val_idx)}')
                train_set = Subset(train_val_set, train_idx)
                val_set = Subset(train_val_set, val_idx)
                # Redirect both stdout and stderr
                with contextlib.redirect_stdout(io.StringIO()), contextlib.redirect_stderr(io.StringIO()):
                    trainer = Trainer(
                        accelerator='gpu',
                        devices=1,
                        max_epochs=n_epochs,
                        enable_progress_bar=False,  # Disable progress bar
                        enable_model_summary=False,
                        logger=False,  # Disable logging
                        enable_checkpointing=False,
                    )

                    # Train model
                    trainer.fit(pretrained_model, train_dataloaders=DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=True),
                                val_dataloaders=DataLoader(val_set, batch_size=batch_size, shuffle=False))

                    # Evaluate the model on the validation set
                    val_result = trainer.validate(pretrained_model, dataloaders=DataLoader(val_set, batch_size=len(val_set), shuffle=False))
                # print(val_result)
                val_scores.append(val_result[0]['val_f1'])  # Optimizing based on validation f1-score
            
            # return the average validation score
            avg_val_f1 = np.mean(val_scores)
            return avg_val_f1

        study = optuna.create_study(direction='maximize')
        study.optimize(objective_cv, n_trials=n_trials)

        # print(f'Best hyperparameters for fold {i}: {study.best_params}')
        best_outer_hyperparameters[i] = study.best_params

        # train the best inner model on the entire train/val set with the best hyperparameters
        best_params = study.best_params
        final_model = ShallowNet(
            fc_layers=best_params['fc_layers'],
            dropout=best_params['dropout'],
            optimizer_name=best_params['optimizer'],
            learning_rate=best_params['learning_rate']
        )

        pretrained_final_model = transfer_freeze_weights(pretrained_weights, final_model, freeze=True)
        
        # Define the checkpoint callback with the fold number in the filename
        fully_trained_callback = ModelCheckpoint(
            filename=f'fold-{i}-last'
        )

        final_trainer = Trainer(
            accelerator='gpu',
            devices=1,
            max_epochs=n_epochs,
            enable_progress_bar=False,
            enable_model_summary=False,
            logger=logger,
            enable_checkpointing=True,
            callbacks=[fully_trained_callback],
        )
        final_trainer.fit(
            pretrained_final_model,
            train_dataloaders=DataLoader(train_val_set, batch_size=best_params['batch_size'], shuffle=True), # maybe drop_last=True??
        )

        # evaluate the model on the test set
        test_result = final_trainer.test(pretrained_final_model, dataloaders=DataLoader(test_set, batch_size=len(test_set), shuffle=False))
        print(f'Test result for fold {i}: {test_result}')
        test_scores.append(test_result[0]['test_f1'])
    
    return test_scores, best_outer_hyperparameters, fully_trained_callback.dirpath

In [4]:
def pipeline_tuning(n_epochs, n_trials, dataset, pretext_task, weights_dir=r'D:\Documents\Master_Data_Science\Thesis\thesis_code\DataScience_Thesis\data\pretext_model_weights'):
    '''
    Function to perform the pipeline for hyperparameter tuning
    :param n_epochs: number of epochs to train the model
    :param n_trials: number of trials for the hyperparameter optimization
    :param dataset: the dataset containing the EEG epochs
    :param: pretext_task: a string indicating the specific pretext task to load the weights from
    :param weights_dir: the directory containing the pretrained weights
    '''
    labels = np.array(dataset.diagnosis)
    groups = np.array(dataset.participant_ids)
    pretrained_weights = torch.load(f'{weights_dir}\{pretext_task}_weights.pt')
    test_scores, best_outer_hyperparameters, checkpoints_path = nested_cv_optuna(n_epochs, n_trials, dataset, labels, groups, pretrained_weights)
    return test_scores, best_outer_hyperparameters, checkpoints_path

## ShallowNet Architecture

In [5]:
class ShallowNet(pl.LightningModule):
        """
        Pytorch implementation of the ShallowNet Encoder.
        Code taken and adjusted from:
        https://github.com/MedMaxLab/selfEEG/blob/024402ba4bde95051d86ab2524cc71105bfd5c25/selfeeg/models/zoo.py#L693
        """

        def __init__(self,
                    fc_layers, # number of fully connected layers
                    learning_rate, # learning rate
                    optimizer_name, # optimizer
                    dropout, # dropout probability
                    num_classes=5, # number of classes in the dataset
                    samples=1244,
                    chans=26, # number of EEG channels
                    F=40, # number of output filters in the temporal convolution layer
                    K1=25, # length of the temporal convolutional layer
                    pool=75, # temporal pooling kernel size
                    num_extracted_features=100, # number of features to extract
                    ):
            super(ShallowNet, self).__init__()
            self.learning_rate = learning_rate
            self.optimizer_name = optimizer_name

            self.conv1 = nn.Conv2d(1, F, (1, K1), stride=(1, 1))
            self.conv2 = nn.Conv2d(F, F, (chans, 1), stride=(1, 1))
            self.batch1 = nn.BatchNorm2d(F)
            self.pool2 = nn.AvgPool2d((1, pool), stride=(1, 15))
            self.flatten2 = nn.Flatten()
            self.drop1 = nn.Dropout(0.2)
            self.lin = nn.Linear(
                F * ((samples - K1 + 1 - pool) // 15 + 1), num_extracted_features
            )

            # part for downstream task
            self.batch2 = nn.BatchNorm1d(num_extracted_features)
            self.fcs = ModuleList()
            self.dropouts = ModuleList()
            input_size = num_extracted_features # Input size for the first layer
            output_size = num_classes # Output size for the last layer
            sizes = np.linspace(input_size, output_size, fc_layers + 1, dtype=int) # Calculate the size for each layer
            for i in range(fc_layers): # Create the layers
                self.fcs.append(Linear(sizes[i], sizes[i + 1]))
                if i < fc_layers - 1:  # Append dropout only if it's not the last layer
                    self.dropouts.append(Dropout(p=dropout))

            # add metrics
            self.train_f1 = F1Score(task='multiclass', num_classes=5, average='macro')
            self.val_f1 = F1Score(task='multiclass', num_classes=5, average='macro')

        def forward(self, x):
            x = self.conv1(x)
            x = self.conv2(x)
            x = self.batch1(x)
            x = torch.square(x)
            x = self.pool2(x)
            x = torch.log(torch.clamp(x, 1e-7, 10000))
            x = self.flatten2(x)
            x = self.drop1(x)
            x = self.lin(x) # These are the SSL features
            x = self.batch2(x) # Standardize the features
            for i, fc in enumerate(self.fcs):
                if i < len(self.fcs) - 1:  # Apply relu only if it's not the last layer
                    x = F.relu(fc(x))
                    x = self.dropouts[i](x)
                else:
                    x = fc(x) # Apply the last layer without relu & no dropout

            return x
        
        def configure_optimizers(self):
            if self.optimizer_name == 'Adam':
                optimizer = torch.optim.Adam(self.parameters(), lr=self.learning_rate)
            elif self.optimizer_name == 'RMSprop':
                optimizer = torch.optim.RMSprop(self.parameters(), lr=self.learning_rate)
            elif self.optimizer_name == 'SGD':
                optimizer = torch.optim.SGD(self.parameters(), lr=self.learning_rate)
            else:
                raise ValueError(f'Unsupported optimizer: {self.optimizer_name}')
            return optimizer
        
        def training_step(self, train_batch, batch_idx):
            x = train_batch[0]
            y = train_batch[1]
            x = x.unsqueeze(1)
            y = y.long()
            logits = self.forward(x)
            loss = F.cross_entropy(logits, y)
            y_hat = torch.argmax(logits, dim=1)
            self.log('train_loss', loss, on_epoch=True, prog_bar=True)
            self.log('train_f1', self.train_f1(y_hat, y), on_epoch=True, prog_bar=True)
            return loss
        
        def validation_step(self, val_batch, batch_idx):
            x = val_batch[0]
            y = val_batch[1]
            x = x.unsqueeze(1)
            y = y.long()
            logits = self.forward(x)
            loss = F.cross_entropy(logits, y)
            y_hat = torch.argmax(logits, dim=1)
            self.log('val_loss', loss, on_epoch=True, prog_bar=True)
            self.log('val_f1', self.val_f1(y_hat, y), on_epoch=True, prog_bar=True)
            return loss
        
        def test_step(self, test_batch, batch_idx):
            x = test_batch[0]
            y = test_batch[1]
            x = x.unsqueeze(1)
            y = y.long()
            logits = self.forward(x)
            loss = F.cross_entropy(logits, y)
            y_hat = torch.argmax(logits, dim=1)
            self.log('test_loss', loss, on_epoch=True, prog_bar=True)
            self.log('test_f1', self.val_f1(y_hat, y), on_epoch=True, prog_bar=True)
            return loss
        
        def predict_step(self, batch):
            x = batch[0]
            x = x.unsqueeze(1)
            logits = self.forward(x)
            y_hat = torch.argmax(logits, dim=1)
            return y_hat

## Loading data

In [6]:

df_participants = pd.read_pickle(r'D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN_participants_V2_data\df_participants.pkl')
sample_df = pd.read_pickle(r'D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TD-BRAIN_extracted_features\df_selected_stat_features.pkl')
sample_ids = sample_df['ID'].unique() # obtain unique IDs from subsampled dataframe containing epoched features
df_sample = df_participants[df_participants['participants_ID'].isin(sample_ids)] # filter participants dataframe to only include subsampled IDs
df_sample = df_sample[df_sample['sessID'] == 1] # filter first session
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(df_sample['diagnosis'])
df_sample['labels'] = le.transform(df_sample['diagnosis'])
# 
print(df_sample.shape)
print(df_sample['diagnosis'].value_counts())



(225, 13)
diagnosis
ADHD       45
HEALTHY    45
MDD        45
OCD        45
SMC        45
Name: count, dtype: int64


In [7]:
# load the epochs into a dataset
participant_ids = df_sample['participants_ID'].tolist()
dataset = EpochDataset(participant_ids, r"D:\Documents\RU\Master_Neurobiology\Internship_jaar_2\Project\TD-BRAIN\TDBRAIN-dataset-derivatives\thesis_epoched_data\EC",
                       df_sample)
print(len(dataset))
print(dataset[0][0].shape)
print(dataset[0][1])
print(dataset[0][2])

No files found for participant ID sub-88073521
Loaded 2688 epochs
2688
torch.Size([26, 1244])
4
sub-87964717


## Tuning

### 1. Within RP pretext task

#### 1a. Tpos = 1

In [ ]:
%%capture # cannot suppress output from pytorch lightning
# perform nested cross-validation with hyperparameter optimization
test_scores, best_outer_hyperparameters, checkpoints_path = pipeline_tuning(
    n_epochs=3,
    n_trials=1,
    dataset=dataset,
    pretext_task='fullytrained_withinRP_tpos1_ShallowNet_pretext_model'
)

In [ ]:
# evaluate tuning results
print(f'Test f1-scores: {test_scores}')
print(f'Average test f1-scores: {np.mean(test_scores)}')
print(f'Std of test f1-scores: {np.std(test_scores)}')
print(f'Best fold test f1-score: {np.max(test_scores)}')

evaluate_best_model(test_scores, best_outer_hyperparameters, checkpoints_path)

#### 1b. Tpos = 2

In [ ]:
%%capture # cannot suppress output from pytorch lightning
# perform nested cross-validation with hyperparameter optimization
test_scores, best_outer_hyperparameters, checkpoints_path = pipeline_tuning(
    n_epochs=3,
    n_trials=1,
    dataset=dataset,
    pretext_task='fullytrained_withinRP_tpos2_ShallowNet_pretext_model'
)

In [ ]:
# evaluate tuning results
print(f'Test f1-scores: {test_scores}')
print(f'Average test f1-scores: {np.mean(test_scores)}')
print(f'Std of test f1-scores: {np.std(test_scores)}')
print(f'Best fold test f1-score: {np.max(test_scores)}')

evaluate_best_model(test_scores, best_outer_hyperparameters, checkpoints_path)

#### 1c. Tpos = 4

In [ ]:
%%capture # cannot suppress output from pytorch lightning
# perform nested cross-validation with hyperparameter optimization
test_scores, best_outer_hyperparameters, checkpoints_path = pipeline_tuning(
    n_epochs=3,
    n_trials=1,
    dataset=dataset,
    pretext_task='fullytrained_withinRP_tpos4_ShallowNet_pretext_model'
)

In [ ]:
# evaluate tuning results
print(f'Test f1-scores: {test_scores}')
print(f'Average test f1-scores: {np.mean(test_scores)}')
print(f'Std of test f1-scores: {np.std(test_scores)}')
print(f'Best fold test f1-score: {np.max(test_scores)}')

evaluate_best_model(test_scores, best_outer_hyperparameters, checkpoints_path)

### 2. Across RP pretext task

#### 2a. Tpos = 1

#### 2b. Tpos = 2

#### 2c. Tpos = 4

### 3. Across Subject pretext task

### 4. Contrastive loss pretext task